# 错误信息汇总处理

# 安装flash-attn 报错

In [ ]:
Requirement already satisfied: mpmath<1.4.0,>=1.1.0 in /root/miniconda3/lib/python3.12/site-packages (from sympy->torch->flash-attn) (1.3.0)
Building wheels for collected packages: flash-attn
  Building wheel for flash-attn (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      
      
      torch.__version__  = 2.3.0+cu121
      
      
      /root/miniconda3/lib/python3.12/site-packages/setuptools/__init__.py:81: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      running bdist_wheel
      Guessing wheel URL:  https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.4.post1/flash_attn-2.7.4.post1+cu12torch2.3cxx11abiFALSE-cp312-cp312-linux_x86_64.whl
      error: Remote end closed connection without response
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: Could not build wheels for flash-attn, which is required to install pyproject.toml-based projects


## 问题分析
依赖问题：flash-attn 依赖于 torch 和其他库，而安装过程中可能由于某些依赖未正确处理导致失败。
依赖冲突：torch 和 flash-attn 的版本不兼容。

## 处理方案
更新flash-attn版本
pip install -U flash-attn --no-build-isolation

## transformers推理报错（一）

In [ ]:
File ~/miniconda3/lib/python3.12/site-packages/transformers/utils/import_utils.py:1951, in _LazyModule._get_module(self, module_name)
   1949     return importlib.import_module("." + module_name, self.__name__)
   1950 except Exception as e:
-> 1951     raise RuntimeError(
   1952         f"Failed to import {self.__name__}.{module_name} because of the following error (look up to see its"
   1953         f" traceback):\n{e}"
   1954     ) from e

RuntimeError: Failed to import transformers.models.qwen2_5_omni.modeling_qwen2_5_omni because of the following error (look up to see its traceback):
No module named 'triton'

## 解决方案
根据错误信息: RuntimeError: Failed to import transformers.models.qwen2_5_omni.modeling_qwen2_5_omni because of the following error (look up to see its traceback):
No module named 'triton'
可以看出来，缺少了triton模块
```bash
pip install triton
```


## transformers推理报错（二）

In [ ]:
File ~/miniconda3/lib/python3.12/site-packages/qwen_omni_utils/v2_5/audio_process.py:46, in process_audio_info(conversations, use_audio_in_video)
     42 assert _check_if_video_has_audio(
     43     path
     44 ), "Video must has audio track when use_audio_in_video=True"
     45 if path.startswith("http://") or path.startswith("https://"):
---> 46     audios.append(librosa.load(audioread.ffdec.FFmpegAudioFile(path), sr=16000)[0])
     47 elif path.startswith("file://"):
     48     audios.append(librosa.load(path[len("file://") :], sr=16000)[0])

File ~/miniconda3/lib/python3.12/site-packages/audioread/ffdec.py:152, in FFmpegAudioFile.__init__(self, filename, block_size)
    142     self.proc = popen_multiple(
    143         COMMANDS,
    144         ['-i', filename, '-f', 's16le', '-'],
   (...)
    148         creationflags=PROC_FLAGS,
    149     )
    151 except OSError:
--> 152     raise NotInstalledError()
    154 finally:
    155     # Reset previous error mode on Windows. (We can change this
    156     # back now because the flag was inherited by the subprocess;
    157     # we don't need to keep it set in the parent process.)
    158     if windows:

NotInstalledError:

### 问题分析
1. **错误位置**：
   - 错误发生在 `qwen_omni_utils/v2_5/audio_process.py` 文件的第 46 行，具体代码为：
     ```python
     audios.append(librosa.load(audioread.ffdec.FFmpegAudioFile(path), sr=16000)[0])
     ```
   - 这行代码尝试通过 `librosa` 和 `audioread` 加载音频文件。`audioread` 使用 `ffmpeg` 作为后端来处理音频文件。

2. **错误原因**：
   - `audioread` 在尝试调用 `ffmpeg` 时，发现 `ffmpeg` 未安装或未正确配置，因此抛出了 `NotInstalledError` 异常。

3. **依赖关系**：
   - `audioread` 是一个用于读取音频文件的库，它依赖于 `ffmpeg` 来处理音频文件。
   - `librosa` 是一个音频处理库，它内部使用 `audioread` 来加载音频文件。

### 解决方法
#### 1. 安装 `ffmpeg`
确保 `ffmpeg` 已正确安装并可以在系统中使用。以下是安装方法：

- **在 Linux 上安装**：
  ```bash
  sudo apt-get update
  sudo apt-get install ffmpeg
  ```

- **在 macOS 上安装**：
  ```bash
  brew install ffmpeg
  ```

- **在 Windows 上安装**：
  1. 下载 `ffmpeg` 的预编译版本：[FFmpeg Download](https://ffmpeg.org/download.html)
  2. 解压下载的文件，并将 `bin` 文件夹路径添加到系统的环境变量 `PATH` 中。

#### 2. 验证 `ffmpeg` 是否安装成功
在终端中运行以下命令，检查 `ffmpeg` 是否正确安装：
```bash
ffmpeg -version
```
如果安装成功，你应该能看到 `ffmpeg` 的版本信息。


#### 3. 确保 `audioread` 和 `librosa` 使用正确的 `ffmpeg` 路径
如果 `ffmpeg` 已安装，但仍然报错，可能是因为 `audioread` 或 `librosa` 无法找到 `ffmpeg` 的路径。可以通过以下方法解决：

- **设置环境变量**：
  在终端中运行以下命令，设置 `ffmpeg` 的路径（假设 `ffmpeg` 已安装在 `/usr/bin/ffmpeg`）：
  ```bash
  export AUDIOREAD_FFMPEG_PATH=/usr/bin/ffmpeg
  ```

- **在代码中指定 `ffmpeg` 路径**：
  如果你无法修改环境变量，可以在代码中显式指定 `ffmpeg` 的路径。例如：
  ```python
  import os
  os.environ['AUDIOREAD_FFMPEG_PATH'] = '/usr/bin/ffmpeg'
  ```

In [ ]:
root@autodl-container-4ed149a141-6d10b153:~# ffmpeg -version
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-lv2 --enable-omx --enable-openal --enable-opencl --enable-opengl --enable-sdl2 --enable-pocketsphinx --enable-librsvg --enable-libmfx --enable-libdc1394 --enable-libdrm --enable-libiec61883 --enable-chromaprint --enable-frei0r --enable-libx264 --enable-shared
libavutil      56. 70.100 / 56. 70.100
libavcodec     58.134.100 / 58.134.100
libavformat    58. 76.100 / 58. 76.100
libavdevice    58. 13.100 / 58. 13.100
libavfilter     7.110.100 /  7.110.100
libswscale      5.  9.100 /  5.  9.100
libswresample   3.  9.100 /  3.  9.100
libpostproc    55.  9.100 / 55.  9.100

# transformers推理报错（三）

In [ ]:
File ~/miniconda3/lib/python3.12/site-packages/transformers/generation/utils.py:1945, in GenerationMixin._prepare_special_tokens(self, generation_config, kwargs_has_attention_mask, device)
   1939     raise ValueError(
   1940         "`decoder_start_token_id` or `bos_token_id` has to be defined for encoder-decoder generation."
   1941     )
   1942 if not is_torchdynamo_compiling():  # Checks that depend on tensor-dependent control flow
   1943     if (
   1944         eos_token_tensor is not None
-> 1945         and isin_mps_friendly(elements=eos_token_tensor, test_elements=pad_token_tensor).any()
   1946     ):
   1947         if kwargs_has_attention_mask is not None and not kwargs_has_attention_mask:
   1948             logger.warning_once(
   1949                 "The attention mask is not set and cannot be inferred from input because pad token is same as "
   1950                 "eos token. As a consequence, you may observe unexpected behavior. Please pass your input's "
   1951                 "`attention_mask` to obtain reliable results."
   1952             )

NotImplementedError: aten::_local_scalar_dense: attempted to run this operator with Meta tensors, but there was no abstract impl or Meta kernel registered. You may have run into this message while using an operator with PT2 compilation APIs (torch.compile/torch.export); in order to use this operator with those APIs you'll need to add an abstract impl.Please see the following doc for next steps: https://docs.google.com/document/d/1_W62p8WJOQQUzPsJYa7s701JXt0qf2OfLub2sbkHOaU/edit

In [ ]:
GPU资源查看如下：

In [ ]:
root@autodl-container-4ed149a141-6d10b153:~/autodl-tmp# nvidia-smi
Wed Apr  2 08:58:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA vGPU-32GB               On  |   00000000:2E:00.0 Off |                  N/A |
| 30%   30C    P8              7W /  320W |   22029MiB /  32760MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|    0   N/A  N/A      1447      C   /root/miniconda3/bin/python                     0MiB |

清理和释放 GPU 资源的步骤（可以先尝试重启内核，不行再进行kill进程）


在 Linux 系统中，清理和释放 GPU 资源通常需要终止占用 GPU 的进程。根据你提供的 `nvidia-smi` 输出，当前有一个 Python 进程（PID 1447）正在运行，但它的 GPU 内存占用为 0 MiB。然而，GPU 的总内存使用达到了 22 GB，这表明可能存在其他未列出的进程占用了 GPU 内存。

以下是清理和释放 GPU 资源的方式：

### 1. **查找并终止占用 GPU 的进程**

#### 使用 `nvidia-smi` 查找进程
运行以下命令，查找所有占用 GPU 的进程：
```bash
nvidia-smi pmon -s u
```
这个命令会显示更详细的 GPU 进程信息，包括每个进程的内存使用情况。

#### 终止进程
找到占用 GPU 内存的进程后，可以使用 `kill` 命令终止它们。例如：
```bash
kill -9 <PID>
```
将 `<PID>` 替换为实际的进程 ID。

### 2. **清理 GPU 缓存**

#### 使用 PyTorch 清理 GPU 缓存
如果你使用的是 PyTorch，可以在代码中显式清理 GPU 缓存：
```python
import torch
torch.cuda.empty_cache()
```

### 3. 重启系统
如果所有方法都无法释放 GPU 资源，重启系统是最直接的方法：
sudo reboot

In [1]:
# transformers推理报错（四）

In [ ]:
NotInstalledError                         Traceback (most recent call last)
Cell In[3], line 5
      2 video_path = "files/music.mp4"
      4 display(Video(video_path, width=640, height=360))
----> 5 display(Audio(librosa.load(audioread.ffdec.FFmpegAudioFile(video_path), sr=16000)[0], rate=16000))
      7 ## Use a local HuggingFace model to inference.
      8 response, audio  = inference(video_path)

File ~/miniconda3/lib/python3.12/site-packages/audioread/ffdec.py:152, in FFmpegAudioFile.__init__(self, filename, block_size)
    142     self.proc = popen_multiple(
    143         COMMANDS,
    144         ['-i', filename, '-f', 's16le', '-'],
   (...)
    148         creationflags=PROC_FLAGS,
    149     )
    151 except OSError:
--> 152     raise NotInstalledError()
    154 finally:
    155     # Reset previous error mode on Windows. (We can change this
    156     # back now because the flag was inherited by the subprocess;
    157     # we don't need to keep it set in the parent process.)
    158     if windows:


## 问题分析：
`NotInstalledError` 是由 `audioread` 库抛出的异常，通常是因为以下原因之一：
- **FFmpeg 未安装**：`audioread` 依赖 FFmpeg 来处理音频文件，但系统中未安装 FFmpeg。
- **FFmpeg 未正确配置**：即使安装了 FFmpeg，但系统无法找到其可执行文件路径。
- **环境变量问题**：FFmpeg 的路径未添加到系统的环境变量中。

## 解决方案

### **方法 1：检查 FFmpeg 是否安装**
1. 打开终端或命令提示符。
2. 输入以下命令检查 FFmpeg 是否安装：
   ```bash
   ffmpeg -version
   ```
   如果返回 FFmpeg 的版本信息，则说明 FFmpeg 已安装；如果没有返回任何信息或提示命令未找到，则需要安装 FFmpeg。

### **方法 2：安装 FFmpeg**
根据你的操作系统，安装 FFmpeg 的方法如下：

- **Windows**：
  1. 访问 [FFmpeg 官方下载页面](https://ffmpeg.org/download.html)。
  2. 下载适合 Windows 的静态构建版本（通常是一个 `.zip` 文件）。
  3. 解压下载的文件到一个目录，例如 `C:\ffmpeg`。
  4. 将 `C:\ffmpeg\bin` 添加到系统的环境变量中：
     - 右键点击“此电脑”或“我的电脑”，选择“属性”。
     - 点击“高级系统设置”。
     - 在“系统属性”窗口中，点击“环境变量”。
     - 在“系统变量”中找到 `Path`，点击“编辑”。
     - 添加 `C:\ffmpeg\bin` 到变量值中。
     - 点击“确定”保存更改。

- **macOS**：
  1. 使用 Homebrew 安装 FFmpeg：
     ```bash
     brew install ffmpeg
     ```

- **Linux**：
  1. 使用包管理器安装 FFmpeg。例如，在 Ubuntu 或 Debian 上：
     ```bash
     sudo apt update
     sudo apt install ffmpeg
     ```